In [1]:
from sklearn.datasets import load_digits
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [2]:
digits = load_digits()
X, y = digits.data, digits.target

In [3]:
def fit_estimator(estimator):
    return cross_val_score(estimator, X, y, cv=10).mean()

In [4]:
def write_answer(filename, answer):
    with open(filename, "w") as fout:
        fout.write(str(answer))

Создайте DecisionTreeClassifier с настройками по умолчанию и измерьте качество его работы с помощью cross_val_score. Эта величина и будет ответом в пункте 1.

In [5]:
tree = DecisionTreeClassifier(random_state=42)

In [6]:
fit_estimator(tree)

0.8370850802141552

In [7]:
write_answer('answer1.txt', fit_estimator(tree))

Воспользуйтесь BaggingClassifier из sklearn.ensemble, чтобы обучить бэггинг над DecisionTreeClassifier. Используйте в BaggingClassifier параметры по умолчанию, задав только количество деревьев равным 100.

Качество классификации новой модели - ответ в пункте 2. Обратите внимание, как соотносится качество работы композиции решающих деревьев с качеством работы одного решающего дерева.

In [8]:
from sklearn.ensemble import BaggingClassifier

In [9]:
bag = BaggingClassifier(tree, 
                        n_estimators=100, 
                        random_state=42)

In [10]:
fit_estimator(bag)

0.9266223799121667

In [11]:
write_answer('answer2.txt', fit_estimator(bag))

Теперь изучите параметры BaggingClassifier и выберите их такими, чтобы каждый базовый алгоритм обучался не на всех d признаках, а на d\sqrt{d}d
​ случайных признаков. Качество работы получившегося классификатора - ответ в пункте 3. Корень из числа признаков - часто используемая эвристика в задачах классификации, в задачах регрессии же часто берут число признаков, деленное на три. Но в общем случае ничто не мешает вам выбирать любое другое число случайных признаков.

In [12]:
from numpy import sqrt

In [13]:
sqrt_bag = BaggingClassifier(tree, 
                             n_estimators=100, 
                             max_features = int(sqrt(X.shape[1])), 
                             random_state=42)

In [14]:
fit_estimator(sqrt_bag)

0.9311692335245662

In [15]:
write_answer('answer3.txt', fit_estimator(sqrt_bag))

Наконец, давайте попробуем выбирать случайные признаки не один раз на все дерево, а при построении каждой вершины дерева. Сделать это несложно: нужно убрать выбор случайного подмножества признаков в BaggingClassifier и добавить его в DecisionTreeClassifier. Какой параметр за это отвечает, можно понять из документации sklearn, либо просто попробовать угадать (скорее всего, у вас сразу получится). Попробуйте выбирать опять же d\sqrt{d}d
​ признаков. Качество полученного классификатора на контрольной выборке и будет ответом в пункте 4.

In [16]:
sqrt_tree = DecisionTreeClassifier(max_features = int(sqrt(X.shape[1])), 
                                   random_state=42)

final_bag = BaggingClassifier(sqrt_tree, 
                             n_estimators=100, 
                             random_state=42)

In [17]:
fit_estimator(final_bag)

0.9533321022473988

In [28]:
write_answer('answer4.txt', fit_estimator(final_bag))

Полученный в пункте 4 классификатор - бэггинг на рандомизированных деревьях (в которых при построении каждой вершины выбирается случайное подмножество признаков и разбиение ищется только по ним). Это в точности соответствует алгоритму Random Forest, поэтому почему бы не сравнить качество работы классификатора с RandomForestClassifier из sklearn.ensemble. Сделайте это, а затем изучите, как качество классификации на данном датасете зависит от количества деревьев, количества признаков, выбираемых при построении каждой вершины дерева, а также ограничений на глубину дерева. Для наглядности лучше построить графики зависимости качества от значений параметров, но для сдачи задания это делать не обязательно.

На основе наблюдений выпишите через пробел номера правильных утверждений из приведенных ниже в порядке возрастания номера (это будет ответ в п.5)

1) Случайный лес сильно переобучается с ростом количества деревьев

2) При очень маленьком числе деревьев (5, 10, 15), случайный лес работает хуже, чем при большем числе деревьев

3) С ростом количества деревьев в случайном лесе, в какой-то момент деревьев становится достаточно для высокого качества классификации, а затем качество существенно не меняется.

4) При большом количестве признаков (для данного датасета - 40, 50) качество классификации становится хуже, чем при малом количестве признаков (5, 10). Это связано с тем, что чем меньше признаков выбирается в каждом узле, тем более различными получаются деревья (ведь деревья сильно неустойчивы к изменениям в обучающей выборке), и тем лучше работает их композиция.

5) При большом количестве признаков (40, 50, 60) качество классификации лучше, чем при малом количестве признаков (5, 10). Это связано с тем, что чем больше признаков - тем больше информации об объектах, а значит алгоритм может делать прогнозы более точно.

6) При небольшой максимальной глубине деревьев (5-6) качество работы случайного леса намного лучше, чем без ограничения глубины, т.к. деревья получаются не переобученными. С ростом глубины деревьев качество ухудшается.

7) При небольшой максимальной глубине деревьев (5-6) качество работы случайного леса заметно хуже, чем без ограничений, т.к. деревья получаются недообученными. С ростом глубины качество сначала улучшается, а затем не меняется существенно, т.к. из-за усреднения прогнозов и различий деревьев их переобученность в бэггинге не сказывается на итоговом качестве (все деревья преобучены по-разному, и при усреднении они компенсируют переобученность друг-друга).

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
rf = RandomForestClassifier(n_estimators=100, 
                            max_features = int(sqrt(X.shape[1])),
                            random_state=42)

In [21]:
fit_estimator(rf)

0.9533106688723997

In [23]:
n_est = [5, 10, 15, 25, 50, 100, 150, 250, 500]
for n in n_est:
    rf = RandomForestClassifier(n_estimators=n,
                                max_features = int(sqrt(X.shape[1])),
                                random_state=42)
    print('Количество деревьев: ', n, ' Оценка на кросс-валидации: ', fit_estimator(rf))

Количество деревьев:  5  Оценка на кросс-валидации:  0.8837028696742983
Количество деревьев:  10  Оценка на кросс-валидации:  0.9272021217853512
Количество деревьев:  15  Оценка на кросс-валидации:  0.9417178614258364
Количество деревьев:  25  Оценка на кросс-валидации:  0.9477646051431515
Количество деревьев:  50  Оценка на кросс-валидации:  0.9550113864646358
Количество деревьев:  100  Оценка на кросс-валидации:  0.9533106688723997
Количество деревьев:  150  Оценка на кросс-валидации:  0.9483315557062137
Количество деревьев:  250  Оценка на кросс-валидации:  0.9488899494409502
Количество деревьев:  500  Оценка на кросс-валидации:  0.9527585675180102


In [24]:
n_feat = [5, 10, 25, 50]
for n in n_feat:
    rf = RandomForestClassifier(n_estimators=100,
                                max_features = n,
                                random_state=42)
    print('Количество признаков: ', n, ' Оценка на кросс-валидации: ', fit_estimator(rf))

Количество признаков:  5  Оценка на кросс-валидации:  0.9517360193918531
Количество признаков:  10  Оценка на кросс-валидации:  0.9506187511126972
Количество признаков:  25  Оценка на кросс-валидации:  0.9482954929094962
Количество признаков:  50  Оценка на кросс-валидации:  0.9360786339457896


In [25]:
n_depth = [1, 3, 5, 10]
for n in n_feat:
    rf = RandomForestClassifier(n_estimators=100,
                                max_depth=n,
                                random_state=42)
    print('Глубина деревьев: ', n, ' Оценка на кросс-валидации: ', fit_estimator(rf))

Глубина деревьев:  5  Оценка на кросс-валидации:  0.9120576801943712
Глубина деревьев:  10  Оценка на кросс-валидации:  0.948862911477352
Глубина деревьев:  25  Оценка на кросс-валидации:  0.9533106688723997
Глубина деревьев:  50  Оценка на кросс-валидации:  0.9533106688723997


In [26]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
fit_estimator(rf)

0.9533106688723997

Ответ на задание: 2, 3, 4, 7

In [30]:
write_answer('answer5.txt', '2 3 4 7')